In [1]:
import pandas as pd
import datetime as dt
import calendar
import string
import contractions
import unicodedata
import sklearn as sk
import sys

In [4]:
sys.path.insert(1,'../../src/')
import utils.general_tools as gt
import utils.setConfig as sc
import etl.etl as etl

In [ ]:
config = pd.read_csv(f'../../bucket/config/{sc.select_data_folder()}',index_col=0)

* The commented code could be used in future iterations for additional features

In [ ]:
# this code may not be updated use the functions in the src folder
punct_to_remove=string.punctuation
raw_data = pd.read_csv(config.loc['iphone6_tweets_repository'][0])
# print(raw_data.loc[1],'\n\n',dt.datetime.strptime(raw_data.loc[1].date,'%b %d, %Y').weekday())
# weekend_vs_weekend=[]
raw_data=raw_data.sort_values(by='date',ascending=False).reset_index(drop=True)
has_url=[] # technically could be part of feature engineering but because text cleanup reshapes url it is done here
txt=[]

#to sort by date, raw_date is three-letter-month day, year format string rather than datetime
raw_data['date']=pd.to_datetime(raw_data['date'])
raw_data=raw_data.sort_values(by='date',ascending=True).reset_index(drop=True)
raw_data['date']=raw_data['date'].apply(lambda _: dt.datetime.strftime(_,'%b %d, %Y'))

# cleaning text for NLP and checking if tweet has url
for i in range(raw_data.shape[0]):
    # weekend_vs_weekend.append('w_day' if dt.datetime.strptime(raw_data.loc[i].date,'%b %d, %Y').weekday()<5 else 'w_end')
    has_url.append(int('http://' in raw_data.loc[i].text or 'https://' in raw_data.loc[i].text))# checks if tweets has a link
    txt.append(unicodedata.normalize('NFKD',contractions.fix(raw_data.loc[i].text.replace('\n',' ').lower().\
            translate(str.maketrans('','',punct_to_remove)))).encode('ascii','ignore').decode('utf-8','ignore'))# text cleanup 
etl_data=raw_data.copy()
etl_data.drop(columns=['date','text'],inplace=True) #current iteration of the model doesn't use date in feature embedding; in future iterations it should because it enhances the count_perID feature
etl_data['text'] = txt
etl_data['has_url']=has_url
# raw_data['Weekday_vs_Weekend']=weekend_vs_weekend

count_values=etl_data['id'].value_counts(sort=False)
etl_data['tweet_count_perID_fromStart'] = [0]*etl_data.shape[0]
for i in range(etl_data.shape[0]-1,-1,-1):
    etl_data.loc[i,'tweet_count_perID_fromStart']= count_values[etl_data.loc[i,'id']]
    count_values[etl_data.loc[i,'id']]-=1


In [5]:
etl_d=etl.perform_etl()
etl_d.head(10)

,name,id,text,has_url
0,iJustine,@ijustine,let us not forget the new base model iphone 15...,1
1,Shishir,@ShishirShelke1,the night time camera shots from iphone 15 pro...,0
2,Steve Grasso,@grassosteve,apple iphone 15 off to a strong start in china...,0
3,ᴛʀᴏʟʟɪɴɢ ɪꜱ ᴀɴ ᴀʀᴛ,@Trolling_isart,omg i cannot believe how over the top the acti...,0
4,Holly - I like tech,@AnxiousHolly,the iphone 15 pro can reach temperatures of up...,0
5,iPhone Used Genuine ,@iPhone_Genuine,iphone se 202064gb 95black color 300000nbinai...,0
6,South China Morning Post,@SCMPNews,apples iphone 15 gets thumbs up from chinese c...,0
7,Verizon News,@VerizonNews,we were onsite last week at the appleevent to ...,0
8,Anthony,@TheGalox_,is it really that hard for apple to put a prop...,0
9,Apple Cycle,@theapplecycle,in my opinion the iphone x does not look that ...,0


In [6]:
full_dataset=etl.feature_addition(etl_d)
full_dataset.head(10)

,name,id,text,has_url,tweet_count_perID_fromStart
0,iJustine,@ijustine,let us not forget the new base model iphone 15...,1,1
1,Shishir,@ShishirShelke1,the night time camera shots from iphone 15 pro...,0,1
2,Steve Grasso,@grassosteve,apple iphone 15 off to a strong start in china...,0,1
3,ᴛʀᴏʟʟɪɴɢ ɪꜱ ᴀɴ ᴀʀᴛ,@Trolling_isart,omg i cannot believe how over the top the acti...,0,1
4,Holly - I like tech,@AnxiousHolly,the iphone 15 pro can reach temperatures of up...,0,1
5,iPhone Used Genuine ,@iPhone_Genuine,iphone se 202064gb 95black color 300000nbinai...,0,1
6,South China Morning Post,@SCMPNews,apples iphone 15 gets thumbs up from chinese c...,0,1
7,Verizon News,@VerizonNews,we were onsite last week at the appleevent to ...,0,1
8,Anthony,@TheGalox_,is it really that hard for apple to put a prop...,0,1
9,Apple Cycle,@theapplecycle,in my opinion the iphone x does not look that ...,0,1


In [7]:
etl.save_post_etl_and_featureEng(full_dataset)